In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

2023-04-05 14:34:45.194981: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
data = pd.read_csv('balanced_credit.csv')

X = data.drop('default', axis=1).values
y = data['default'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(8, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_split=0.2, epochs=25, batch_size=32, verbose=1)

y_pred = (model.predict(X_test) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Epoch 1/25


2023-04-05 15:50:33.190628: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


266/266 [==============================] - ETA: 0s - loss: 4232.8325 - accuracy: 0.5225

2023-04-05 15:50:39.975135: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

In [10]:
data = pd.read_csv('balanced_credit.csv')

X = data.drop('default', axis=1).values
y = data['default'].values

# Convert binary labels to categorical labels with two classes
y = np.array([[1, 0] if label == 0 else [0, 1] for label in y])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(25, activation='relu'),
    Dense(25, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_split=0.2, epochs=25, batch_size=32, verbose=1)

y_pred = model.predict(X_test).argmax(axis=1)
y_test_labels = y_test.argmax(axis=1)
accuracy = accuracy_score(y_test_labels, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Epoch 1/25


2023-04-05 15:42:04.679333: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


266/266 [==============================] - ETA: 0s - loss: 0.6496 - accuracy: 0.6189

2023-04-05 15:42:12.820396: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


266/266 [==============================] - 9s 32ms/step - loss: 0.6496 - accuracy: 0.6189 - val_loss: 0.6098 - val_accuracy: 0.6756
Epoch 2/25
266/266 [==============================] - 8s 29ms/step - loss: 0.6087 - accuracy: 0.6717 - val_loss: 0.5911 - val_accuracy: 0.6996
Epoch 3/25
266/266 [==============================] - 8s 29ms/step - loss: 0.5954 - accuracy: 0.6869 - val_loss: 0.5919 - val_accuracy: 0.6831
Epoch 4/25
266/266 [==============================] - 8s 29ms/step - loss: 0.5869 - accuracy: 0.6988 - val_loss: 0.5808 - val_accuracy: 0.6973
Epoch 5/25
266/266 [==============================] - 8s 29ms/step - loss: 0.5867 - accuracy: 0.6950 - val_loss: 0.5813 - val_accuracy: 0.6996
Epoch 6/25
266/266 [==============================] - 8s 29ms/step - loss: 0.5791 - accuracy: 0.7015 - val_loss: 0.5799 - val_accuracy: 0.7006
Epoch 7/25
266/266 [==============================] - 8s 29ms/step - loss: 0.5826 - accuracy: 0.6948 - val_loss: 0.5795 - val_accuracy: 0.6992
Epoch 8/25

KeyboardInterrupt: 

In [13]:
import itertools
from tensorflow.keras import backend
from scikeras.wrappers import KerasClassifier
import time

data = pd.read_csv('balanced_credit.csv')

X = data.drop('default', axis=1).values
y = data['default'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def create_model(learning_rate=0.001, neurons=64, dropout_rate=0.2):
    backend.clear_session()
    optimizer = Adam(learning_rate=learning_rate)
    if dropout_rate is None:
        model = Sequential([
            Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)),
            Dense(neurons // 2, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
    else:
        if neurons < 128:
            model = Sequential([
                Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)),
                Dropout(dropout_rate),
                Dense(neurons // 2, activation='relu'),
                Dropout(dropout_rate),
                Dense(1, activation='sigmoid')
            ])
        else:
            model = Sequential([
                Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)),
                Dropout(dropout_rate * 2),
                Dense(neurons // 2, activation='relu'),
                Dropout(dropout_rate * 2),
                Dense(neurons // 4, activation='relu'),
                Dropout(dropout_rate),
                Dense(1, activation='sigmoid')
            ])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Hyperparameter grid
learning_rates = [0.001]
neurons = [16, 32, 64, 128]
dropout_rates = [None, 0.2]
epochs = [25, 50]

combinations = list(itertools.product(learning_rates, neurons, dropout_rates, epochs))

best_accuracy = 0
best_params = None

for combo in combinations:
    start_time = time.time()

    learning_rate, neuron, dropout_rate, epoch = combo
    model = KerasClassifier(model=create_model, epochs=epoch, batch_size=32, verbose=0,
                            learning_rate=learning_rate, neurons=neuron, dropout_rate=dropout_rate)

    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    print(f"Accuracy: {accuracy:.4f} with learning_rate={learning_rate}, neurons={neuron}, dropout_rate={dropout_rate}, epochs={epoch}")

    elapsed_time = (time.time() - start_time) / 60
    print(f"Elapsed time: {elapsed_time:.2f} minutes")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = combo

print(f"Best accuracy: {best_accuracy:.4f} with learning_rate={best_params[0]}, neurons={best_params[1]}, dropout_rate={best_params[2]}, epochs={best_params[3]}")

2023-04-05 15:47:10.363201: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-05 15:49:43.693694: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Accuracy: 0.7009 with learning_rate=0.001, neurons=16, dropout_rate=None, epochs=25
Elapsed time: 2.57 minutes


2023-04-05 15:49:44.411136: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 